In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import imageio
from wrappers.caiman_wrapper import *
from wrappers.dummy_wrapper import *
from wrappers.data_wrapper import *

### dummy

In [ ]:
file_path = os.path.join(
    '/Users', 'shogoakiyama', 'caiman_data', 
    'example_movies', 'Sue_2x_3000_40_-46.tif')

In [ ]:
image = ImageData(file_path, 'dummy')

In [ ]:
dummy_image2image(ImageData(file_path), opts={'a': 1})

In [ ]:
def func1():
    print(sys._getframe().f_code.co_name)
func1()

### caiman

In [ ]:
info = {}
file_path = os.path.join(
    '/Users', 'shogoakiyama', 'caiman_data', 
    'example_movies', 'Sue_2x_3000_40_-46.tif')

In [ ]:
info['caiman_mc'] = caiman_mc(file_path)
info['caiman_cnmf'] = caiman_cnmf(info['caiman_mc']['images'])

In [ ]:
info['caiman_mc']

In [ ]:
plt.imshow(info['caiman_mc']['images'][0])

In [ ]:
plt.imshow(info['caiman_mc']['total_template_rig'])

In [ ]:
# plot
plt.imshow(info['caiman_mc']['total_template_rig'], cmap = 'gray')

In [ ]:
# shifts rig
plt.plot(info['caiman_mc']['shifts_rig'])

In [ ]:
# calcium image
plot_contours_nb(info['caiman_mc']['images'], info['caiman_cnmf']['cnm'])

### suite2p

In [ ]:
from wrappers.suite2p_wrapper import run_s2p, roi
info = {}

file_path = os.path.join(
    '/Users', 'shogoakiyama', 'caiman_data', 
    'example_movies', 'Sue_2x_3000_40_-46.tif')

In [ ]:
info['run_s2p'] = run_s2p(file_path)
info['roi'] = roi(info['run_s2p']['save_path'], info['run_s2p']['Lx'], info['run_s2p']['Ly'])

In [ ]:
plt.figure(figsize=(16, 4))

plt.subplot(1, 4, 1)
plt.imshow(info['run_s2p']['max_proj'], cmap='gray')
plt.title("Registered Image, Max Projection")

plt.subplot(1, 4, 2)
plt.imshow(np.nanmax(info['roi']['im'], axis=0), cmap='jet')
plt.title("All ROIs Found")

plt.subplot(1, 4, 3)
plt.imshow(np.nanmax(info['roi']['im'][~info['roi']['iscell']], axis=0, ), cmap='jet')
plt.title("All Non-Cell ROIs")

plt.subplot(1, 4, 4)
plt.imshow(np.nanmax(info['roi']['im'][info['roi']['iscell']], axis=0), cmap='jet')
plt.title("All Cell ROIs");